## Inference Operator PySDK E2E Expereience (JumpStart model)

In [ ]:
from sagemaker.hyperpod import list_clusters, set_cluster_context

In [ ]:
list_clusters(region='us-east-2')

In [ ]:
# choose the HP cluster
set_cluster_context('<my-cluster>', region='us-east-2')

### Select a JumpStart model from public hub

In [ ]:
# Import the helper module
from sagemaker.hyperpod.inference.jumpstart_public_hub_visualization_utils import get_all_public_hub_model_data

# Load and display SageMaker public hub models
get_all_public_hub_model_data(region="us-east-2")

### Create JumpStart model endpoint

#### Create from spec object

In [ ]:
from sagemaker.hyperpod.inference.config.hp_jumpstart_endpoint_config import Model, Server,SageMakerEndpoint, TlsConfig, EnvironmentVariables
from sagemaker.hyperpod.inference.hp_jumpstart_endpoint import HPJumpStartEndpoint
from sagemaker.hyperpod.common.config.metadata import Metadata
import yaml
import time

<b>Note:</b> We auto-generate config class definitions above using script, such as `Model`, `Server`, `SageMakerEndpoint`. This is based on [Inference CRD file](https://code.amazon.com/packages/AWSCrescendoInferenceOperator/blobs/mainline/--/dist/config/crd/inference.sagemaker.aws.amazon.com_jumpstartmodels.yaml).

In [ ]:
# If you don't set metadata name, it will be default to endpoint name
# If you don't set namespace, it will be default to "default"
metadata=Metadata(name='<metadata_name>', namespace='<namespace>')

# create configs
model=Model(
    model_id='deepseek-llm-r1-distill-qwen-1-5b'
)
server=Server(
    instance_type='ml.g5.8xlarge',
)
endpoint_name=SageMakerEndpoint(name='<my-endpoint-name>')

# create spec
js_endpoint=HPJumpStartEndpoint(
    metadata=metadata,
    model=model,
    server=server,
    sage_maker_endpoint=endpoint_name
)

In [ ]:
# use spec to deploy
js_endpoint.create()

In [ ]:
# poll status
t = 0
timeout = 600  # 600 seconds timeout  
interval = 15  # poll every 15 seconds

while t < timeout:
    # use refresh to fetch latest status
    js_endpoint.refresh()

    try:
        # deployment status will be available immediately
        deployment_status = js_endpoint.status.deploymentStatus.deploymentObjectOverallState
        if deployment_status== 'DeploymentFailed':
            print('Deployment failed!')
            break

        # endpoint status will appear be available from refresh() at some point
        endpoint_status = js_endpoint.status.endpoints.sagemaker.state
        if endpoint_status == 'CreationCompleted':
            print('Endpoint is available!')
            break
    except:
        pass

    time.sleep(interval)
    t += interval
    print('Refreshing instance status...')

if t >= timeout:
    print('Endpoint creation timed out!')

In [ ]:
js_endpoint.status.endpoints.sagemaker.state

In [ ]:
# print endpoint in yaml
def print_yaml(endpoint):
    print(yaml.dump(endpoint.model_dump(exclude_none=True)))

In [ ]:
# output is similar to kubectl get jumpstartmodels
endpoint_list = HPJumpStartEndpoint.list()
print_yaml(endpoint_list[0])

In [ ]:
# output is similar to kubectl describe jumpstartmodel
endpoint = HPJumpStartEndpoint.get(name='<my-endpoint-name>')
print_yaml(endpoint)

### Invoke endpoint

In [ ]:
# invoke
data='{"inputs":"What is the capital of USA?"}'

endpoint.invoke(body=data).body.read()

In [ ]:
# get operator logs
print(js_endpoint.get_operator_logs(since_hours=0.1))

In [ ]:
# delete endpoint
endpoint.delete()